In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import numpy as np
import multiprocessing

In [2]:
# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("DEVICE",device)

DEVICE cuda


## Model

In [3]:
v = 0     # model version
in_c = 2  # number of input channels
num_c = 1 # number of classes to predict

In [4]:
# The optical flow input will look like this
of = torch.randn(1,2,1164,874)

In [5]:
model = EfficientNet.from_pretrained(f'efficientnet-b{v}', in_channels=in_c, num_classes=num_c)
model.to(device);

Loaded pretrained weights for efficientnet-b0


#### The output of the model will look like this

In [6]:
of = of.to(device)
model(of).item()

0.0030444227159023285

## Data

In [7]:
# directory with the optical flow images
of_dir = 'C:/Users/Cana/Documents/GitHub/pyslam/outFlow0'
# labels as txt file
labels_f = 'C:/Users/Cana/Documents/GitHub/pyslam/labelsPITCH.txt'
model_file='C:/Users/Cana/Documents/GitHub/pyslam/outputPITCH.pth'

In [8]:
class OFDataset(Dataset):
    def __init__(self, of_dir, label_f):
        self.len = len(list(Path(of_dir).glob('*.npy')))
        print("len",self.len)
        self.of_dir = of_dir
        self.label_file = open(label_f).readlines()
        #print("selflavelfike",self.label_file)
    def __len__(self): return self.len
    def __getitem__(self, idx):

        of_array = np.load(Path(self.of_dir)/f'{idx}.npy')
  
     
     
        of_tensor = torch.squeeze(torch.tensor(of_array))
      
        label = float(self.label_file[idx].split()[0])
        #print("label",label)
        return [of_tensor, label]

In [9]:
ds = OFDataset(of_dir, labels_f)
print("DS1",ds[0][1])


len 5995
DS1 0.03149205029088487


In [10]:
# 80% of data for training
# 20% of data for validation
train_split = .8

In [11]:
ds_size = len(ds)
indices = list(range(ds_size))
split = int(np.floor(train_split * ds_size))
train_idx, val_idx = indices[:split], indices[split:]

In [12]:

sample = ds[3]
assert type(sample[0]) == torch.Tensor
assert type(sample[1]) == float

In [13]:
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)


In [14]:
cpu_cores = multiprocessing.cpu_count()
cpu_cores

24

In [15]:
print("ds",ds)
train_dl = DataLoader(ds, batch_size=3, sampler=train_sampler, num_workers=0)
val_dl = DataLoader(ds, batch_size=3, sampler=val_sampler, num_workers=0)
print("train_dl",train_dl)

state = torch.load(model_file)
model.load_state_dict(state.state_dict())

ds <__main__.OFDataset object at 0x000001DB0150DDF0>
train_dl <torch.utils.data.dataloader.DataLoader object at 0x000001DB015299D0>


<All keys matched successfully>

## Train

In [16]:
epochs = 1
log_train_steps = 1

In [17]:
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters())

In [18]:
for epoch in range(epochs):
    print(f'Epoch ${epoch}')
    model.train()
    running_loss = 0.0
    for i, sample in enumerate(train_dl):
        of_tensor = sample[0].cuda()       
        label = sample[1].float().cuda()
        #print("Labelx",label)
        opt.zero_grad()
        pred = torch.squeeze(model(of_tensor))   
        loss = criterion(pred, label)
        loss.backward()
        opt.step()
        print(f'i: {i}')
        if(i>500):
            break
    print(f'Pred {label}={pred}')
    # validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for j, val_sample in enumerate(val_dl):
            of_tensor = val_sample[0].cuda()
            label = val_sample[1].float().cuda()
            pred = torch.squeeze(model(of_tensor))           
            loss = criterion(pred, label)
            val_losses.append(loss)
        print(f'{epoch}: {sum(val_losses)/len(val_losses)}')
torch.save(model, model_file)

Epoch $0
